In [ ]:
"""Notebook for debugging bootstrap CIs."""
import numpy as np
import pandas as pd
import plotly.express as px

from thesis.classes import Instrument, LocalATEs
from thesis.simple_model.funcs import _true_late
from thesis.simple_model.task_simple_model_sims import ID_TO_KWARGS, LATES_COMPLIER

In [ ]:
n_obs = 10_000
complier = -0.1
boot_met = "numerical_delta"

In [ ]:
path_to_res = [
    value.path_to_data
    for key, value in ID_TO_KWARGS.items()
    if value.n_obs == n_obs
    and value.local_ates.complier == complier
    and value.bootstrap_method == boot_met
]

In [ ]:
res = pd.read_pickle(path_to_res[0])  # noqa: S301
res["covers_true_param"] = (res["lo"] <= res["true"]) & (res["hi"] >= res["true"])
res.head()

In [ ]:
res.mean()

In [ ]:
instrument = Instrument(
    support=np.array([0, 1]),
    pmf=np.array([0.5, 0.5]),
    pscores=np.array([0.4, 0.6]),
)

lates = [
    LocalATEs(
        never_taker=0,
        complier=late_complier,
        always_taker=1,
    )
    for late_complier in LATES_COMPLIER
]

true_params = [
    _true_late(u_hi=0.2, instrument=instrument, local_ates=local_lates)
    for local_lates in lates
]

In [ ]:
# Plot true_params against LATES_COMPLIER
data = pd.DataFrame(
    {
        "true_param": true_params,
        "complier": LATES_COMPLIER,
    },
)

fig = px.line(data, x="complier", y="true_param")
fig.show()